# split_check - Notebook de uso (replica do original)

Este notebook reproduz os passos do notebook original: divisão por paciente, verificações de vazamento (patient/study/filestem/duplicatas/temporal), e preparação do dataset no formato YOLO.

ATENÇÃO: este notebook pressupõe que o arquivo `dataset.csv` foi obtido do site oficial do dataset. Por favor baixe o dataset no site oficial antes de executar (não inclua dados sensíveis no repositório público).

## 1) Preparação do ambiente e imports
Descomente a linha de instalação se precisar instalar dependências a partir de `requirements.txt`.

In [1]:
# Instale dependências (descomente se necessário)
# %pip install -r requirements.txt

import sys
from pathlib import Path
import importlib.util

# Garantir que o projeto está no sys.path para imports locais
proj_root = Path('.').resolve()
if str(proj_root) not in sys.path:
    sys.path.insert(0, str(proj_root))
print('Projeto em', proj_root)

# Importar o módulo novo (split_check)
import split_check as sc
# Configure logging uma vez para o notebook (prevenindo handlers duplicados)
try:
    sc.configure_logging()
except Exception:
    # configure_logging pode não existir em versões antigas do módulo; ignorar sem quebrar
    pass
print('split_check versão (módulo):', getattr(sc, '__version__', 'n/a')) if hasattr(sc, '__version__') else print('split_check importado')

Projeto em C:\Users\rodri\Desktop\GRAZPEDWRI-DX-Split
split_check importado


## 2) Caminhos e configuração
Ajuste os caminhos abaixo conforme sua estrutura local.

In [2]:
CSV_PATH = Path('14825193/dataset.csv')
OUTPUT_DIR = Path('splits')
PATIENT_COL = 'patient_id'
STUDY_COL = 'study_number'
DO_ORGANIZE = True  # mudar para True quando quiser copiar imagens/labels
IMG_DIR = Path('images')
LBL_DIR = Path('labels')
DST_ROOT = Path('dataset_yolo')
SEED = 42

print('CSV_PATH:', CSV_PATH)
print('OUTPUT_DIR:', OUTPUT_DIR)
print('Images dir:', IMG_DIR, '\nLabels dir:', LBL_DIR)

CSV_PATH: 14825193\dataset.csv
OUTPUT_DIR: splits
Images dir: images 
Labels dir: labels


## 3) Carregar dataset
Tenta carregar o CSV do dataset; se não encontrado, avisa e interrompe (não gera dados sintéticos automaticamente aqui, para evitar confusão em produção).

In [3]:
import pandas as pd

try:
    df = sc.load_dataset(CSV_PATH)
except FileNotFoundError as e:
    print('❌', e)
    print('Por favor baixe o dataset no site oficial e coloque em', CSV_PATH)
    raise

# Exibir primeiras linhas e estatísticas básicas
display(df.head(3))
print(f'Total amostras: {len(df)} | pacientes únicos: {df[PATIENT_COL].nunique() if PATIENT_COL in df.columns else "N/A"}')

INFO: Dataset carregado: 20327 amostras, 17 colunas


,filestem,patient_id,study_number,timehash,gender,age,laterality,projection,initial_exam,ao_classification,cast,diagnosis_uncertain,osteopenia,fracture_visible,metal,pixel_spacing,device_manufacturer
0,0001_1297860395_01_WRI-L1_M014,1,1,1297860395,M,14.1,L,1,1.0,23r-M/2.1,NaN,NaN,NaN,NaN,NaN,0.144,Siemens
1,0001_1297860435_01_WRI-L2_M014,1,1,1297860435,M,14.1,L,2,1.0,23r-M/2.1,NaN,NaN,NaN,1.0,NaN,0.144,Siemens
2,0002_0354485735_01_WRI-R1_F012,2,1,354485735,F,12.0,R,1,1.0,23r-M/2.1,NaN,1.0,NaN,NaN,NaN,0.144,Siemens


Total amostras: 20327 | pacientes únicos: 6091


## 4) Dividir por paciente (sem vazamento) e salvar splits
Usamos a função `split_by_patient` do módulo para obter `train/val/test` e salvamos os CSVs.

In [4]:
# Realizar split
train_df, val_df, test_df = sc.split_by_patient(df, patient_col=PATIENT_COL, seed=SEED)

# Salvar splits
sc.save_splits(train_df, val_df, test_df, out_dir=OUTPUT_DIR, patient_col=PATIENT_COL)


INFO: Split feito: train=14170, val=4118, test=2039 samples
INFO: Splits salvos em splits


## 5) Verificações (patient, study, filestem, duplicatas, temporal)
As verificações abaixo seguem a lógica do notebook original. Algumas checagens apenas imprimem avisos; outras lançam erro se vazamento for encontrado.

In [5]:
# Checagens simples por chaves
keys_to_check = [k for k in [PATIENT_COL, 'study_number', 'filestem', 'filepath'] if k in df.columns]
print('Chaves a checar:', keys_to_check)
sc.check_group_overlap(train_df, val_df, test_df, keys_to_check)

INFO: [patient_id] train∩val=0, train∩test=0, val∩test=0


INFO: [study_number] train∩val=11, train∩test=9, val∩test=9
INFO: [filestem] train∩val=0, train∩test=0, val∩test=0


Chaves a checar: ['patient_id', 'study_number', 'filestem']


In [6]:
# Checagens simples por chaves
keys_to_check = [k for k in [PATIENT_COL, 'study_number', 'filestem', 'filepath'] if k in df.columns]
print('Chaves a checar:', keys_to_check)
sc.check_group_overlap(train_df, val_df, test_df, keys_to_check)

# Checagem por estudo (se existir) — garante study_key disjuntas
if 'study_number' in df.columns:
    make_study_key = lambda d: (d[PATIENT_COL].astype(str) + '#' + d['study_number'].astype(str))
    try:
        sc.check_disjoint(make_study_key(train_df), make_study_key(val_df), 'study_key train∩val')
        sc.check_disjoint(make_study_key(train_df), make_study_key(test_df), 'study_key train∩test')
        sc.check_disjoint(make_study_key(val_df), make_study_key(test_df), 'study_key val∩test')
        print('✅ Nenhum vazamento por study_key detectado (se aplicável)')
    except AssertionError as e:
        print('❌ Vazamento detectado em study_key:', e)

# Duplicatas exatas (ignorar colunas não relevantes)
ignore_cols = [c for c in ['patient_id','study_number','timehash','filepath'] if c in df.columns]
sc.check_exact_row_duplicates(train_df, val_df, test_df, ignore_cols=ignore_cols)

# Verificação temporal para até 3 colunas candidatas
candidate_dates = [c for c in df.columns if any(k in c.lower() for k in ['date','time','timestamp','timehash'])]
if candidate_dates:
    for c in candidate_dates[:3]:
        sc.check_temporal_leakage(train_df, val_df, test_df, datetime_col=c)
else:
    print('Nenhuma coluna temporal detectada.')

INFO: [patient_id] train∩val=0, train∩test=0, val∩test=0
INFO: [study_number] train∩val=11, train∩test=9, val∩test=9
INFO: [filestem] train∩val=0, train∩test=0, val∩test=0
INFO: study_key train∩val: 0 interseções
INFO: study_key train∩test: 0 interseções
INFO: study_key val∩test: 0 interseções
INFO: [duplicatas exatas] train∩val=0, train∩test=0, val∩test=0
INFO: [temporal timehash] max(train)=2011-12-22 01:12:28 | min(val)=1972-06-22 10:11:39 | min(test)=1973-07-13 06:21:15


Chaves a checar: ['patient_id', 'study_number', 'filestem']
✅ Nenhum vazamento por study_key detectado (se aplicável)


## 6) Organização YOLO (copiar imagens/labels) — opcional
Mude `DO_ORGANIZE = True` acima para realmente copiar os arquivos.

In [7]:
if DO_ORGANIZE:
    try:
        sc.organize_yolo(splits_dir=OUTPUT_DIR, src_images=IMG_DIR, src_labels=LBL_DIR, dst_root=DST_ROOT)
        print('✅ Organização YOLO concluída em', DST_ROOT)
    except Exception as e:
        print('Erro ao organizar YOLO:', e)
else:
    print('Organização YOLO desabilitada (DO_ORGANIZE=False)')

INFO: 📂 Movendo arquivos do split train (14170 amostras)
INFO: 📂 Movendo arquivos do split val (4118 amostras)
INFO: 📂 Movendo arquivos do split test (2039 amostras)
INFO: ✅ Organização YOLO concluída em dataset_yolo
INFO: Copied original meta.yaml from meta.yaml to dataset_yolo\original_meta.yaml
INFO: Updated data.yaml written to dataset_yolo\data.yaml


✅ Organização YOLO concluída em dataset_yolo
